In [1]:
import tensorflow as tf 
import tensorflow_probability as tfp

In [2]:
n = 5
m = 3
num_chains = 7

In [3]:
kernel = tf.random.normal(shape=(num_chains, n, m))
bias = tf.random.normal(shape=(num_chains, m))

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-01-01 23:43:38.950745: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-01 23:43:38.951520: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
kernel

<tf.Tensor: shape=(7, 5, 3), dtype=float32, numpy=
array([[[-0.14952676, -1.3622406 ,  1.0941813 ],
        [ 1.3325045 , -0.23650269,  2.3725414 ],
        [ 2.6410306 ,  1.4768419 ,  0.8744571 ],
        [ 1.7699282 ,  1.0301566 , -0.5623233 ],
        [ 0.7204717 ,  1.426441  ,  0.01340317]],

       [[-1.1981047 , -1.6441406 , -1.2258801 ],
        [-0.56674504,  0.55103743, -0.965394  ],
        [ 0.36918512,  0.42778346,  0.10250291],
        [ 0.01525468,  1.370953  , -0.85353136],
        [-0.9551967 ,  0.71005726, -0.07376222]],

       [[ 1.274002  , -1.089386  ,  0.8338626 ],
        [-0.9021529 , -1.8194867 ,  0.34940085],
        [-0.8090955 ,  0.8523396 ,  2.128027  ],
        [ 0.8624403 , -1.5313218 , -0.46443266],
        [ 0.21712169,  0.69957006, -0.04429394]],

       [[ 1.1489364 , -0.25453055, -0.17664196],
        [ 0.80244774, -0.3206635 , -2.5002396 ],
        [-1.3042488 , -0.5276184 , -0.83282226],
        [-1.1054032 , -0.04637325, -0.02209354],
        [-0.

In [5]:
mcmc_samples = [kernel, bias]
def get_posterior_mcmc(mcmc_samples):
    def posterior_mcmc(kernel_size, bias_size, dtype=None):
        return tfp.layers.DistributionLambda(
            lambda t: tfp.distributions.Empirical(mcmc_samples)
        )
    return posterior_mcmc

In [6]:
posterior = get_posterior_mcmc(mcmc_samples=mcmc_samples)

In [57]:
kernel_posterior = tfp.distributions.Empirical(samples=kernels, event_ndims=2)
bias_posterior = tfp.distributions.Empirical(samples=bias, event_ndims=1)


In [58]:
print(kernel_posterior.sample())
print(kernel)


tf.Tensor(
[[-0.04243673  0.7222836   1.3690245 ]
 [ 0.82769895  0.42433393  1.7027127 ]
 [-0.00542013  1.6417382  -0.436844  ]
 [ 0.00722046 -0.5278278  -1.1738304 ]
 [-1.3716816  -1.7135777  -0.45039752]], shape=(5, 3), dtype=float32)
tf.Tensor(
[[[-0.14952676 -1.3622406   1.0941813 ]
  [ 1.3325045  -0.23650269  2.3725414 ]
  [ 2.6410306   1.4768419   0.8744571 ]
  [ 1.7699282   1.0301566  -0.5623233 ]
  [ 0.7204717   1.426441    0.01340317]]

 [[-1.1981047  -1.6441406  -1.2258801 ]
  [-0.56674504  0.55103743 -0.965394  ]
  [ 0.36918512  0.42778346  0.10250291]
  [ 0.01525468  1.370953   -0.85353136]
  [-0.9551967   0.71005726 -0.07376222]]

 [[ 1.274002   -1.089386    0.8338626 ]
  [-0.9021529  -1.8194867   0.34940085]
  [-0.8090955   0.8523396   2.128027  ]
  [ 0.8624403  -1.5313218  -0.46443266]
  [ 0.21712169  0.69957006 -0.04429394]]

 [[ 1.1489364  -0.25453055 -0.17664196]
  [ 0.80244774 -0.3206635  -2.5002396 ]
  [-1.3042488  -0.5276184  -0.83282226]
  [-1.1054032  -0.04637325

In [59]:
print(bias_posterior.sample())
bias

tf.Tensor([-2.57398     0.4472205  -0.91244274], shape=(3,), dtype=float32)


<tf.Tensor: shape=(7, 3), dtype=float32, numpy=
array([[-0.21160358, -0.40113866, -0.16318968],
       [-2.57398   ,  0.4472205 , -0.91244274],
       [-1.1954323 , -0.55392605,  0.7614507 ],
       [ 0.07599503,  0.2532692 , -1.4473243 ],
       [ 1.0263864 , -0.48374212,  0.45287538],
       [ 1.7757125 ,  0.4919715 ,  1.2057395 ],
       [-0.78445745, -0.5498506 , -0.03055292]], dtype=float32)>

In [70]:
weights = [
    [kernel[i], bias[i]] for i in range(kernel.shape[0])
]

In [84]:
weight_posterior = tfp.distributions.JointDistributionSequential([
    tfp.distributions.Empirical(samples=kernel, event_ndims=2),
    tfp.distributions.Empirical(samples=bias, event_ndims=1),
])

In [85]:
weight_posterior.sample()

[<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[ 1.274002  , -1.089386  ,  0.8338626 ],
        [-0.9021529 , -1.8194867 ,  0.34940085],
        [-0.8090955 ,  0.8523396 ,  2.128027  ],
        [ 0.8624403 , -1.5313218 , -0.46443266],
        [ 0.21712169,  0.69957006, -0.04429394]], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.21160358, -0.40113866, -0.16318968], dtype=float32)>]

In [86]:
print(kernel)
print(bias)

tf.Tensor(
[[[-0.14952676 -1.3622406   1.0941813 ]
  [ 1.3325045  -0.23650269  2.3725414 ]
  [ 2.6410306   1.4768419   0.8744571 ]
  [ 1.7699282   1.0301566  -0.5623233 ]
  [ 0.7204717   1.426441    0.01340317]]

 [[-1.1981047  -1.6441406  -1.2258801 ]
  [-0.56674504  0.55103743 -0.965394  ]
  [ 0.36918512  0.42778346  0.10250291]
  [ 0.01525468  1.370953   -0.85353136]
  [-0.9551967   0.71005726 -0.07376222]]

 [[ 1.274002   -1.089386    0.8338626 ]
  [-0.9021529  -1.8194867   0.34940085]
  [-0.8090955   0.8523396   2.128027  ]
  [ 0.8624403  -1.5313218  -0.46443266]
  [ 0.21712169  0.69957006 -0.04429394]]

 [[ 1.1489364  -0.25453055 -0.17664196]
  [ 0.80244774 -0.3206635  -2.5002396 ]
  [-1.3042488  -0.5276184  -0.83282226]
  [-1.1054032  -0.04637325 -0.02209354]
  [-0.3546948  -0.89654654 -0.24649477]]

 [[-0.5644232  -0.6285046   0.8852413 ]
  [-0.6983524   0.11108248 -0.51431894]
  [ 0.55166274  0.8976841   0.41962337]
  [ 0.6292436  -0.20004515  0.51020414]
  [-0.20997182 -0.750

In [87]:
print(weight_posterior)

tfp.distributions.JointDistributionSequential("JointDistributionSequential", batch_shape=[[], []], event_shape=[[5, 3], [3]], dtype=[float32, float32])


In [93]:
kernel_prior = tfp.distributions.Normal(loc=0., scale=1.)
bias_prior = tfp.distributions.Normal(loc=0., scale=1.)
weight_prior = tfp.distributions.JointDistributionSequential([
    kernel_prior,
    bias_prior
])

In [99]:
layer = tfp.layers.DenseVariational(
    units=bias.shape[1],
    make_posterior_fn=lambda kernel_size, bias_size, dtype: weight_posterior,
    make_prior_fn=lambda kernel_size, bias_size, dtype: weight_prior
)

In [100]:
print(layer)

In [101]:
x = tf.random.normal(shape=(4, 3))

In [103]:
model = tf.keras.Sequential([layer])

2022-01-02 00:38:37.769310: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [104]:
model(x)

TypeError: Exception encountered when calling layer "dense_variational_1" (type DenseVariational).

'JointDistributionSequential' object is not callable

Call arguments received:
  • inputs=tf.Tensor(shape=(4, 3), dtype=float32)

In [105]:
checkpointer = tf.train.Checkpoint()

In [106]:
checkpointer

In [108]:
checkpointer = tf.train.Checkpoint(weight_posterior)

In [109]:
checkpointer.save("./joint_pdf")

ValueError: Unable to save the object {-1: ListWrapper([<tfp.distributions.Empirical 'Empirical' batch_shape=[] event_shape=[5, 3] dtype=float32>, <tfp.distributions.Empirical 'Empirical' batch_shape=[] event_shape=[3] dtype=float32>])} (a dictionary wrapper constructed automatically on attribute assignment). The wrapped dictionary contains a non-string key which maps to a trackable object or mutable data structure.

If you don't need this dictionary checkpointed, wrap it in a non-trackable object; it will be subsequently ignored.